In [ ]:
!pip install transformers

In [2]:
from transformers import ConvBertTokenizer, ConvBertForMaskedLM
import torch

In [3]:
def predict_next_word(sentence):
    # Load ConvBERT tokenizer and model
    tokenizer = ConvBertTokenizer.from_pretrained('YituTech/conv-bert-base')
    model = ConvBertForMaskedLM.from_pretrained('YituTech/conv-bert-base')

    # Tokenize input sentence
    tokens = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt')

    # Find the masked token
    masked_index = torch.where(tokens == tokenizer.mask_token_id)[1].tolist()[0]

    # Generate predictions for the masked token
    with torch.no_grad():
        outputs = model(tokens)
        predictions = outputs.logits[0, masked_index]

    # Get the top-k predicted tokens and their probabilities
    top_predictions = torch.topk(predictions, k=5)
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_predictions.indices.tolist())
    probabilities = top_predictions.values.exp().tolist()

    return predicted_tokens, probabilities

In [5]:
sentence = "I want to [MASK] a car"

predicted_tokens, probabilities = predict_next_word(sentence)
for token, prob in zip(predicted_tokens, probabilities):
    print(f"Token: {token}, Probability: {prob}")

Some weights of ConvBertForMaskedLM were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['generator_predictions.LayerNorm.bias', 'generator_predictions.dense.weight', 'generator_predictions.dense.bias', 'generator_lm_head.bias', 'generator_predictions.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Token: pinyin, Probability: 237.00958251953125
Token: vida, Probability: 156.7217559814453
Token: ##sund, Probability: 150.42002868652344
Token: intersection, Probability: 123.30018615722656
Token: transactions, Probability: 117.59205627441406
